In [5]:
from textblob import TextBlob 
from datetime import datetime
from pythainlp import word_tokenize
from nltk.corpus import stopwords
import nltk
import langdetect
import tweepy as tw
import pandas as pd
import re
import glob
import os
import schedule
import time
import requests
import shutil
from pythainlp.corpus import thai_stopwords

In [6]:
filename = glob.glob('*.csv')

In [9]:
nltk.download('stopwords')  
class DataManager:
    def __init__(self):
        ##-------------------- twitter --------------------##
        self._url = "https://api.aiforthai.in.th/ssense"                     
        self._headers = {'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"}
        consumer_key = "EaFU9nJw2utR0lo2PUmJE3VZy"
        consumer_secret = "DsZuVw0tEl6GHhyK08tunsOE9ICSfwplEhRDMQwB8VIqngZ6i8"
        access_token = "759317188863897600-nuwQmcYfDX8lvdRyw2eCD6fMRMkLzzZ"
        access_token_secret = 'zFFc5OJywNMBrRAblI7kFV62ZTZPHfTU1Q5kZ1cKzUupD'
        auth = tw.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        self._api = tw.API(auth, wait_on_rate_limit=True)

        self._url = "https://api.aiforthai.in.th/ssense"                     
        self._headers = {'Apikey': "0kFkiFLdf4TAyY3JeUT9WVnB5naP6SjW"}

        self.keys = []
        self.df = None
        self._start = 0
        self.filenames = []

    def getSentimentENG(self,text):
        if TextBlob(text).sentiment.polarity > 0:
            return 'positive'
        elif TextBlob(text).sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def getSentimentTH(self,text):
        text = re.sub(r'[%]',' ',text)
        params = {'text':text}
        response = requests.get(self._url, headers=self._headers, params=params)
        try:
            polarity = str(response.json()['sentiment']['polarity'])
        except (KeyError):
            polarity = 'neutral'
        return polarity

    def formatdatetime(self,column):
        self.df[column] = pd.to_datetime(self.df[column]).dt.strftime('%Y/%m/%d') #dmY ทีหลัง
        self.df[column] = pd.to_datetime(self.df[column])
    
    def sortdf(self,columns):
        self.df.sort_values(by=columns,inplace=True)
        return self.df

    def unionfile(self,filenames):              #type filename -> list
        self._start = 0
        self.filenames = filenames
        for file in filenames:
            df1 = pd.read_csv(file)
            if self._start != 0:
                self.df = pd.concat([self.df,df1])
                self.df.drop_duplicates(keep='last',inplace=True)
            else:
                self.df = df1
                self._start += 1
        self.keys = list(set(self.df['Keyword'].tolist()))
        self.collectfile()
        return self.df
    
    def setnewDF(self,dataframe):
        self.df = dataframe
        return self.df
    
    def setdefaultDF(self):
        self.df = self.unionfile(self.filenames)
        return self.df
    
    def collectfile(self):
        self.df["Time"] = pd.to_datetime(self.df["Time"]).dt.strftime('%Y-%m-%d')
        keys = list(set(df['Keyword'].tolist()))
        folder = "collectkeys"
        if not os.path.exists(folder):
            os.mkdir(folder)    
        for key in keys:
            path = str(folder+'/'+key)
            dff = self.df.loc[self.df['Keyword'].isin([key])]
            days = list(set(dff['Time'].tolist()))
            if not os.path.exists(path):
                os.mkdir(path)
            for d in days:
                dfff = dff.loc[dff['Time'].isin([d])]
                dfff.to_csv(path+'/'+key+'_'+d+'.csv',encoding='utf-8',index=False)
    
    def newUnion(self,filenames):
        self.filenames = filenames
        path=os.getcwd()
        keys = []
        start = 0
        for f in glob.glob(path+'/collectkeys/*'):
            keyname = os.path.split(f)[-1]
            keys.append(keyname)
        for k in keys:
            for file in glob.glob(path+'/collectkeys/'+k+'/*.csv'):
                if start == 0:
                    df = pd.read_csv(file)
                    start +=1
                else:
                    dff = pd.read_csv(file)
                    df = pd.concat([df,dff])
        self.keys = keys
        self.collectfile()
        return self.df
    
    def getperiod(self,since,until):  ####column for twitter
        self.formatdatetime('Time')
        dff = self.df
        dff.sort_values(by=['Time','Keyword'],inplace=True)
        if since == None and until != None:
            mask = (dff['Time']<=until)
        elif since != None and until == None:
            mask = (dff['Time']>=since)
        elif since != None and until != None:
            mask = (dff['Time']>=since) & (dff['Time']<=until)
        else:
            return
        return dff.loc[mask]
    
    def getrowwithkeys(self,keys):              #type keys -> list
        df = self.df
        return df.loc[df['Keyword'].isin(keys)]

    def collectwords(self,dataframe):
        #print(dataframe)
        dataframe = dataframe.reset_index()
        th_stopwords = list(thai_stopwords())
        en_stops = set(stopwords.words('english'))
        word = {}
        for index,row in dataframe.iterrows():    #only tweet
            if row['Language'] == 'eng':
                allwords = row['Tweet'].split()
                for w in allwords: 
                    if w not in en_stops:
                        if w in word:
                            word[w] += 1
                        else:
                            word[w] = 1
            elif row['Language'] == 'th':
                allwords = word_tokenize(row['Tweet'], engine='newmm')
                for w in allwords: 
                    if w not in th_stopwords:
                        if w in word:
                            word[w] += 1
                        else:
                            word[w] = 1
            else:
                pass
        if 'RT' in word:
            del word['RT']  #for twitter
        if ' ' in word:
            del word[' ']   #for thai language
        sortword = sorted(word.items(),key=lambda x:x[1],reverse=True)
        worddf = pd.DataFrame(sortword,columns=['Word','Count'])
        return worddf   #word dataframe
        #return sortword     #tuple in list
    
    def deletekeyword(self,keyword):
        path=os.getcwd()
        for k  in keyword:
            shutil.rmtree(path+'//collectkeys//'+k+'//')
            self.keys.remove(k)
            self.df.drop(self.df[self.df['Keyword']==k].index,inplace = True)
        return self.df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
dm = DataManager()
dm.unionfile(filename)

,Keyword,User,Tweet,Language,Time,User Location,Hashtag,Polarity,Likes,Retweet,Sentiment
1,2d animation,LinearUwu,RT blutack1966 The future will always be brigh...,en,2022-04-01,NaN,[],positive,0.0,21.0,0.516667
2,2d animation,animationjobs,2D Background Artist job in Dicesol Animation ...,en,2022-04-01,Global 🌍,[],positive,1.0,1.0,0.211039
4,2d animation,l3oAlways,RT irshema LGGGUYS ALWAYS TELL U SURE DEAL LET...,en,2022-04-01,Poland 🇵🇱,"['LGG', 'websitedesign', 'GraphicDesign', 'vid...",positive,0.0,33.0,0.500000
5,2d animation,BiraboneyeVicky,RT irshema LGGGUYS ALWAYS TELL U SURE DEAL LET...,en,2022-04-01,NaN,"['LGG', 'websitedesign', 'GraphicDesign', 'vid...",positive,0.0,27.0,0.500000
6,2d animation,vedavasai,Totem Creative Mumbai based animation studio w...,en,2022-04-01,"vasai, Mumbai","['vedavasai', 'animation', 'Studio', 'INFO']",positive,2.0,1.0,0.500000
...,...,...,...,...,...,...,...,...,...,...,...
10397,อนิเมะแนะนำ,JokerJokeE7,RT thisisoleang สายวายต้องกรี๊ด เค้ามีกิจกรรมใ...,th,2022-03-30,NaN,[],neutral,0.0,192.0,0.000000
10398,อนิเมะแนะนำ,AnimeICU,แนะนำ AKEBIchan SonoBisqueDollWaKoiWoSuru Part...,th,2022-04-05,NaN,"['AKEBI_chan', 'SonoBisqueDollWaKoiWoSuru']",positive,5.0,1.0,66.670000
10399,อนิเมะแนะนำ,Ruttapak2,RT museacgth ก่อนที่อนิเมะ บันทึกการเดินทางต่า...,th,2022-04-03,NaN,['บ'],neutral,0.0,10.0,0.000000
10400,อนิเมะแนะนำ,Creamyyy_mm,RT thisisoleang สายวายต้องกรี๊ด เค้ามีกิจกรรมใ...,th,2022-04-03,NaN,[],neutral,0.0,192.0,0.000000


In [15]:
from pythainlp.tokenize import word_tokenize
text = "ดูทีวี"
list_word = word_tokenize(text)
print(list_word)

from pythainlp.corpus import thai_stopwords
stopwords = list(thai_stopwords())
list_word_not_stopwords = [i for i in list_word if i not in stopwords]
print(list_word_not_stopwords)

['ดู', 'ทีวี']
['ดู', 'ทีวี']


In [12]:
worddf = dm.collectwords(dm.df)

In [13]:
worddf.head(20)

,Word,Count
0,อนิเมะ,14814
1,แนะนำ,8700
2,ดู,7752
3,ซี,6312
4,Bilibili,6165
5,เค้า,6084
6,โค,6075
7,สาย,6056
8,วาย,6017
9,ฟรี,5987


In [ ]:
dm.deletekeyword(['anime'])

In [ ]:
dm.collectfile()

In [ ]:
import shutil
path=os.getcwd()
shutil.rmtree(path+'//collectkeys//')

In [ ]:
df1 = dm.getrowwithkeys(['shounen']).sort_values('Keyword')
df1

In [ ]:
dm.setnewDF(df1)
dm.getperiod('2022-03-21','2022-03-25')

In [ ]:
dm.setdefaultDF()

In [ ]:
for word in dm.collectword()[:10]:
    print(word)

In [ ]:
df = pd.DataFrame(dm.collectword()[:100],columns=['Word','Count'])
df

In [ ]:
dm = DataManager()
dm.unionfile(filename)
dm.formatdatetime('Time')
print(dm.df['Time'].min().strftime('%Y/%m/%d'))
print(dm.df['Time'].max().strftime('%Y/%m/%d'))

In [ ]:
keys = ['bl anime','anime comedy','anime romance','ต่างโลก','anime','animation','shounen','pixar',
        'harem','fantasy anime','sport anime','attack on titan','disney animation','animation studio',
        'shounen ai','shoujo','อนิเมะ','2d animation','อนิเมะแนะนำ','japan animation']
len(keys)

In [ ]:
import pandas as pd
import re
import glob
filename = ['tweet_data_2032022.csv','tweet_data_1932022.csv','tweet_data_2132022.csv','tweet_data_2232022.csv']
i = 0
for file in filename:
    df1 = pd.read_csv(file)
    if i != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        i += 1

df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%Y/%m/%d')
#df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%d %m %Y')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['Time','Keyword'],inplace=True)

#time = list(set(df['Time'].tolist()))
mask = (df['Time']>='2022-03-19') & (df['Time']<='2022-03-21')
#mask = (df['Time']>='19-03-2022') & (df['Time']<='21-03-2022')
df.loc[mask]

In [ ]:
keyword = ""
keyword = list(map(lambda x: x.lower(), keyword))
keyword
print(type(None))

In [4]:
from pythainlp.corpus import thai_stopwords
th_stopwords = list(thai_stopwords())

en_stops = set(stopwords.words('english'))
filename = ['tweet_data_2732022.csv']
i = 0
for file in filename:
    df1 = pd.read_csv(file)
    if i != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        i += 1
        
dff = df.loc[df['Keyword'].isin(['ต่างโลก','อนิเมะ'])]
dff = dff.reset_index()##
word = {}

for index,row in dff.iterrows():##
    if row['Language'] == 'en':
        allwords = row['Tweet'].split()
        for w in allwords: 
            if w not in en_stops:
                if w in word:
                    word[w] += 1
                else:
                    word[w] = 1
    elif row['Language'] == 'th':   #
        allwords = word_tokenize(row['Tweet'], engine='newmm')  #
        for w in allwords: 
            if w not in th_stopwords:
                if w in word:
                    word[w] += 1
                else:
                    word[w] = 1
    else:
        pass
del word['RT']
del word[' ']
sortword = sorted(word.items(),key=lambda x:x[1],reverse=True)
for i in sortword[:10]:
    print(i)

('อนิเมะ', 1007)
('ทิ้ง', 504)
('นิยาย', 503)
('เรื่อง', 456)
('ตลาดนัด', 392)
('โลก', 377)
('ตอน', 296)
('แฟนตาซี', 274)
('คน', 254)
('ดู', 232)


In [ ]:
dff = dff.reset_index()
for index,row in dff.iterrows():
    print(row['Tweet'],row['Language'])

In [ ]:
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

all_words = ['There', 'is', 'a', 'tree','near','the','river']
for word in all_words: 
    if word not in en_stops:
        print(word)

In [ ]:
import requests
 
url = "https://api.aiforthai.in.th/ssense"
 
text = 'สาขานี้พนักงานน่ารักให้บริการดี'
 
data = {'text':text}
 
headers = {
    'Apikey': "vIQAf35aRkc7QUbR1fTPvzvtkqtSKAaz"
    }
 
response = requests.post(url, data=data, headers=headers)
 
print(response.json())
response.json()['preprocess']['keyword']

In [ ]:
key = list(set(df['Keyword'].tolist()))

In [ ]:
dm = DataManager()
df = dm.unionfile(filename)
df

In [ ]:
df["Time"] = pd.to_datetime(df["Time"]).dt.strftime('%Y-%m-%d')
keys = list(set(df['Keyword'].tolist()))
folder = "collectkeys"

if not os.path.exists(folder):
    os.mkdir(folder)

for key in keys:
    path = str(folder+'/'+key)
    dff = df.loc[df['Keyword'].isin([key])]
    days = list(set(dff['Time'].tolist()))
    if not os.path.exists(path):
        os.mkdir(path)
    for d in days:
        dfff = dff.loc[dff['Time'].isin([d])]
        dfff.to_csv(path+'/'+key+'_'+d+'.csv',encoding='utf-8',index=False)